In [1]:
import json
import os
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
print(KEY)

sk-proj-k4kAaGKYBv7Eopu6bC3BT3BlbkFJiYS8DUgamYJohmBI9xiC


In [6]:
llm  = ChatOpenAI(api_key = KEY,model_name = "gpt-3.5-turbo", temperature=0.5)

In [7]:
from langchain.callbacks import get_openai_callback
from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain
from langchain.llms import openai
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE = '''
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to\
create a quiz of {number} multiple choice question for {subject} in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforiming the text too.
Ensure to make format your response like RESPONSE_JSON below and use it as guide.\
Ensure to make {number} MCQs
{response_json}'''

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE    
)

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [14]:
review_chain= LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain = SequentialChain(
  chains=[quiz_chain, review_chain],
  input_variables=["text", "number", "subject", "tone","response_json"],
  output_variables=["quiz", "review"],
  verbose=True,
)

In [16]:
file_path = r"D:\NLP\MCQGEN\data.txt"

In [17]:
with open(file_path,'r')as file:
    TEXT = file.read()

In [18]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [19]:
NUMBER=5 

SUBJECT="Machine Learning"

TONE="simple"

In [20]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine Learning tutorial covers basic and advanced concepts, specially designed to cater to both students and experienced working professionals.
This machine learning tutorial helps you gain a solid introduction to the fundamentals of machine learning and explore a wide range of techniques, including supervised, unsupervised, and reinforcement learning.
Machine learning (ML) is a subdomain of artificial intelligence (AI) that focuses on developing systems that learnâ€”or improve performanceâ€”based on the data they ingest. Artificial intelligence is a broad word that refers to systems or machines that resemble human intelligence. Machine learning and AI are frequently discussed together, and the terms are occasionally used interchangeably, although they do not signify the same thing. A crucial distinction is that, while all machine learning is AI, not all AI is machine learning.

In [21]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1439
Prompt Tokens:993
Completion Tokens:446
Total Cost:0.0023815000000000004


In [22]:
response

{'text': 'Machine Learning tutorial covers basic and advanced concepts, specially designed to cater to both students and experienced working professionals.\nThis machine learning tutorial helps you gain a solid introduction to the fundamentals of machine learning and explore a wide range of techniques, including supervised, unsupervised, and reinforcement learning.\nMachine learning (ML) is a subdomain of artificial intelligence (AI) that focuses on developing systems that learnâ€”or improve performanceâ€”based on the data they ingest. Artificial intelligence is a broad word that refers to systems or machines that resemble human intelligence. Machine learning and AI are frequently discussed together, and the terms are occasionally used interchangeably, although they do not signify the same thing. A crucial distinction is that, while all machine learning is AI, not all AI is machine learning.\nMachine Learning is the field of study that gives computers the capability to learn without be

In [23]:
quiz=response.get("quiz")

In [24]:
quiz=json.loads(quiz)

In [25]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [26]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: A subdomain of artificial intelligence that focuses on developing systems that learn based on the data they ingest | b: A type of computer programming that requires explicit instructions for every task | c: A technology that gives computers the ability to think and reason like humans | d: A type of robotic technology used in manufacturing processes',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a type of machine learning?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Reinforcement learning | d: Explicit learning',
  'Correct': 'd'},
 {'MCQ': 'What is the main goal of machine learning?',
  'Choices': 'a: To make computers think and reason like humans | b: To develop systems that can learn and improve performance based on data | c: To replace human intelligence with artificial intelligence | d: To create robots that can perform tasks without any programming',
  'Correct': 'b'},
 {'MCQ': 'What di

In [27]:
quiz=pd.DataFrame(quiz_table_data)

In [28]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')